In [11]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import networkx as nx
import json

In [12]:
soup1 = BeautifulSoup(open('data/test.html'))
people1 = soup1.find_all("div", class_="person".split())
soup2 = BeautifulSoup(open('data/test2.html'))
people2 = soup2.find_all("div", class_="person".split())
soup3 = BeautifulSoup(open('data/test3.html'))
people3 = soup3.find_all("div", class_="person".split())
soup4 = BeautifulSoup(open('data/test4.html'))
people4 = soup4.find_all("div", class_="person".split())

print "I found {lp} people".format(lp=len(people1))
print "I found {lp} people".format(lp=len(people2))
print "I found {lp} people".format(lp=len(people3))
print "I found {lp} people".format(lp=len(people4))
print "That's",len(people1)+len(people2)+len(people3)+len(people4),"Out of 517"
#print people3[-1]

I found 213 people
I found 81 people
I found 80 people
I found 143 people
That's 517 Out of 517


In [13]:
def isphone(s):
    if not s.strip():
        return False
    for i in s.strip():
        if not i in '()-0123456789 ':
            return False
    return True
class SimplePerson(dict):
    def __hash__(self):
        """Be careful not to mutate after you've hashed, obviously"""
        return hash(tuple(sorted(self.items())))
    def __init__(self,s):
        self['name'] = s['name']
        self['department'] = s['department']
        self['spouse'] = s['spouse']
        self['displayname'] = ''
    def __str__(self):
        result = '{name} ({department})'.format(name=self['name'],
                                                department=self['department'])
        return result

class Person(dict):
    def __hash__(self):
        """Be careful not to mutate after you've hashed, obviously"""
        return hash(tuple(sorted(self.items())))
    def __init__(self,s):
        self['name'] = s.find_all("div",class_="name".split())[0].get_text().strip()
        d = list(s.find_all("p",class_="departmental".split())[0].strings)
        self['title'] = d[0]
        self['department'] = d[1]
        self.process_professional(s)
        self.process_personal(s)
    def process_personal(self,s):
        self['home_address'] = ''
        self['home_phone'] = ''
        self['spouse'] = ''
        self['children'] = ''
        try:
            d = list(s.find_all("p",class_="personal".split())[0].strings)
            self['home_address'] = d[0].replace('Home:','').strip()
            if isphone(d[1].strip()):
                self['home_phone'] = d[1].strip()
        except IndexError:
            self['home_address'] = ''
            self['home_phone'] = ''
        for dd in d:
            if dd.startswith('Spouse'):
                self['spouse'] = dd.replace('Spouse:','').strip()
            elif dd.startswith('Children'):
                self['children'] = dd.replace('Children:','').strip()
        
    def process_professional(self,s):
        p = s.find_all("p",class_="professional".split())
        self['office'], self['phone'], self['address'], self['email'] = '','','',''
        d = list(p[0].strings)
        parts = [i.strip() for i in d[0].split(',')]
        if len(parts) == 2:
            self['office'], self['phone'] = parts
        elif len(parts) > 2:
            self['phone'] = parts[-1]
            self['office'] = ', '.join(parts[:-1])
        else:
            raise Error('whoa + parts={parts}'.format(parts=parts))
        if len(d) >= 2:
            self['address'] = d[1].split(',')[0].strip()
        if len(d) >= 3:
            self['email'] = d[2].strip()
        # Data corruption
        if 'Bethany Drawer' in self['office']:
            parts = self['office'].split('Bethany Drawer')
            self['office'] = parts[0]
            self['address'] = 'Bethany Drawer ' + parts[1]
        if not '@' in self['email']:
            for pp in p[0].strings:
                if '@' in pp:
                    self['email'] = pp
        if not isphone(self['phone']):
            self['phone'] = ''

    def __repr__(self):
        result = 'Name: {name}\nDepartment: {department}\nTitle: {title}'.format(name=self['name'],
                                                               department=self['department'],
                                                               title=self['title'])
        result += "\nOffice: {office}\nAddress: {address}\nEmail:{email}".format(office=self['office'],
                                                                                address=self['address'],
                                                                                email=self['email'])
        result += '\nPhone: {phone}'.format(phone=self['phone'])
        result += '\nHome Address: {ha}\nHome Phone: {hp}'.format(ha=self['home_address'],
                                                hp = self['home_phone'])
        result += '\nSpouse: {sp}\nChildren: {c}'.format(sp=self['spouse'],
                                                         c=self['children'])
        return result
    def __str__(self):
        result = '{name} ({department})'.format(name=self['name'],
                                                department=self['department'])
        return result
#print Person(people[13+18+2+2])
#print
#print people[13]
trouble = []
success = []
for people in people1, people2, people3, people4:
    for p in people:
        try:
            pp = Person(p)
            # Known data errors
            fixes = {'Heather Lerner':'Biology/Joseph Moore Museum',
                     'Bobbi Cayard-Roberts':'Mathematics/Physics/Computer Science'}
            for (name,dept) in fixes.iteritems():
                if pp['name'] == name:
                    pp['department'] = dept
            success.append(pp)
        except:
            trouble.append(p)
print "I had trouble with {lt} and success with {ls} (should be 517)".format(lt=len(trouble),ls=len(success))
print pp
simple_success = [SimplePerson(p) for p in success]

I had trouble with 0 and success with 517 (should be 517)
Brian Zimmerman (Marketing and Communications)


In [14]:
people = simple_success
def find_dupes(people):
    found = {}
    for p in people:
        if p['name'] in found:
            print "Dupe:"
            print p
        else:
            found[p['name']] = True
    print "Done"
find_dupes(people)

def deptdict(people):
    bydept = {}
    for p in people:
        for d in p['department'].split('/'):
            if d not in bydept:
                bydept[d] = []
            bydept[d].append(p)
    return bydept
def department_pairs(people,kind='circle'):
    bydept = deptdict(people)
    if kind == 'circle':
        # Let's make a circle!
        for (d,members) in bydept.iteritems():
            for i in range(len(members) - 1):
                yield (members[i],members[i+1])
            yield (members[0],members[-1])
    elif kind == 'web':
        for (d,members) in bydept.iteritems():
            for (i,m1) in enumerate(members):
                for m2 in members[i+1:]:
                    yield (m1,m2)
def add_displaynames(people):
    bydept = deptdict(people)
    for (dept,members) in bydept.iteritems():
        members[0]['displayname'] = dept

def spouse_pairs(people):
    for (i,p1) in enumerate(people):
        for p2 in people[i+1:]:
            if p1['spouse'] and p2['spouse']:
                if (p1['spouse'] == p2['name']) or (p2['spouse'] == p1['name']):
                    yield(p1,p2)
                elif p1['spouse'] in p2['name'] and p2['spouse'] in p1['name']:
                    yield(p1,p2)
            else:
                p1last = p1['name'].split()[-1]
                p2last = p2['name'].split()[-1]
                if p1['spouse'] and (p1['spouse'] in p2['name'] and p1last == p2last):
                    yield(p1,p2)
                elif p2['spouse'] and (p2['spouse'] in p1['name'] and p2last == p1last):
                    print "Match6: {p1} {p2}".format(p1=p1['name'],p2=p2['name'])
                    yield(p1,p2)
g = nx.Graph()
add_displaynames(people) # Do this first, because you're using dicts as keys below
for p in people:
    g.add_node(p,department=p['department'])
for (p1,p2) in department_pairs(people,kind='web'):
    g.add_edge(p1,p2)
    g[p1][p2]['colleague']=True
for (p1,p2) in spouse_pairs(people):
    g.add_edge(p1,p2)
    g[p1][p2]['spouse']=True



Done


In [15]:
#nx.draw(g)

In [16]:
# I don't know how to make networkx give me
# a list of edges with integer indices, so 
# I use this to make the JSON in a format that
# D3.js will understand.
intmap = {}
for (i,n) in enumerate(g.node):
    intmap[n] = i

In [17]:
# http://nbviewer.ipython.org/github/ipython-books/cookbook-code/blob/master/notebooks/chapter06_viz/04_d3.ipynb
linklist = [{'source':intmap[n1],
             'target':intmap[n2],
             'value': {True:.9,False:.6}['spouse' in g[n1][n2]]} for (n1,n2) in g.edges()]
#spouselinklist = [{'source':intmap[n1],'target':intmap[n2]} for (n1,n2) in g.edges() if 'spouse' in g[n1][n2]]
with open('graph.json', 'w') as f:
    json.dump({'nodes': [{'name': i['name'],
                          'displayname': i['displayname'],
                          'spouse': i['spouse'],
                          'department': i['department'],
                          'numneighbors': len(g.neighbors(i))}
                              for i in g.nodes()],
               'links': linklist,
               #'spouselinks': spouselinklist
               },
              f, indent=4,)


In [18]:
%%html
<div id="d3-example"></div>
<style>
.node {stroke: #fff; stroke-width: 2.5px;}
.link {stroke: #999; stroke-opacity: .3;}
.darklink {stroke: #0000FF; stroke-opacity: .9; stroke-width: 5.0px}
text {
  font: 15px sans-serif;
  pointer-events: none;
}
</style>

In [19]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.
    
    // First, we specify the size of the canvas containing
    // the visualization (size of the <div> element).
    var width = 1200,
        height = 1200;

    // We create a color scale.
    var color = d3.scale.category10();

    // We create a force-directed dynamic graph layout.
    var force = d3.layout.force()
        .charge(-60)
        .linkDistance(60)
        .gravity(.15)
        .size([width, height]);

    // In the <div> element, we create a <svg> graphic
    // that will contain our interactive visualization.
    var svg = d3.select("#d3-example").select("svg")
    if (svg.empty()) {
        svg = d3.select("#d3-example").append("svg")
                    .attr("width", width)
                    .attr("height", height);
    }
        
    // We load the JSON file.
    d3.json("graph.json", function(error, graph) {
        // In this block, the file has been loaded
        // and the 'graph' object contains our graph.
        
        // We load the nodes and links in the force-directed
        // graph.
        force.nodes(graph.nodes)
            .links(graph.links)
            .start();

        // We create a <line> SVG element for each link
        // in the graph.
        // Some help taken from
        // http://bl.ocks.org/d3noob/5155181
        // .attr("class", "link")
        var link = svg.selectAll(".link")
            .data(graph.links)
            .enter().append("line")
            .attr("class", function(d) {
                if (d.value <= 0.6) {return "link";}
                else {return "darklink"}
                }
            );
        /*var link = svg.selectAll(".link")
            .data(graph.links)
            .enter().append("line")
            .attr("class", "link")
            .style("stroke-width", function(d) { return d.value; });*/

        // We create a <circle> SVG element for each node
        // in the graph, and we specify a few attributes.
        /*var node = svg.selectAll(".node")
            .data(graph.nodes)
            .enter().append("circle")
            .attr("class", "node")
            .attr("r", 7)  // radius
            .style("fill", function(d) {
                // The node color depends on the department.
                return color(d.department); 
            })
            .call(force.drag);
        // The name of each node is the node number.
        node.append("title")
            .text(function(d) { return d.name; });
        // Why doesn't this work to always display text?
        node.append("text")
            .attr("dx", ".10em")
            .attr("dy", ".10em")
            .text(function(d) { return d.name; });*/
        var gnodes = svg.selectAll('g.gnode')
            .data(graph.nodes)
            .enter()
            .append('g')
            .classed('gnode',true);
        var node = gnodes.append("circle")
            .attr("class","node")
            .attr("r",7)
            .style("fill",function(d) {return color(d.department); })
            .call(force.drag);
        var titles = gnodes.append("title")
            .style("text-anchor", "middle")
            .text(function(d) {return d.name});
        var labels = gnodes.append("text")
          .text(function(d) { return d.displayname; });
        
        // We bind the positions of the SVG elements
        // to the positions of the dynamic force-directed graph,
        // at each time step.
        force.on("tick", function() {
            link.attr("x1", function(d) { return d.source.x; })
                .attr("y1", function(d) { return d.source.y; })
                .attr("x2", function(d) { return d.target.x; })
                .attr("y2", function(d) { return d.target.y; });

        // Since we're moving more than just the circle, we
        // need to transform the whole node rather than
        // Just setting cx and cy.
        gnodes.attr("transform", function(d) { 
            return 'translate(' + [d.x, d.y] + ')'; 
        });

        });
    });
});
